# check Gaussian-Fermion package

## import the package

In [2]:
from fermion_gaussian import *

In [9]:
import numpy as np
from numpy import linalg as LA

## Function: correlation_mat(M)
Given a free fermion Hamiltonian taking the following form,
\begin{equation}
H ~=~ \frac{i}{2}\,\sum_{i,j=1}^{2L}\,M_{ij}\,\gamma_i\,\gamma_j
\label{equ:H}
\end{equation}
Input: the matrix $M$

Output: the correlation matrix for the ground state of $H$

In [4]:
def Ising_mat(L):
    # H_Ising = i/2 sum_{k} [ gamma_k gamma_{k+1} - gamma_{k+1} gamma_k ] with periodic boundary condition gamma_{2L+1} = - gamma_1
    M_ij = np.zeros([2*L, 2*L])
    for k in range(2*L-1):
        M_ij[k,k+1] = 1
        M_ij[k+1,k] = -1
    M_ij[2*L-1,0] = -1
    M_ij[0,2*L-1] = 1
    return M_ij

def Ising_correlation_mat(L):
    C_ij = np.zeros([2*L,2*L])
    for i in range(2*L):
        for j in range(i+1, 2*L, 2):
            C_ij[i,j] = 1/(np.sin(np.pi/(2*L)*(i-j))*L)
            C_ij[j,i] = - C_ij[i,j]
    return C_ij

In [6]:
correlation_mat(Ising_mat(L = 3))

array([[ 0.        , -0.66666667,  0.        , -0.33333333,  0.        ,
        -0.66666667],
       [ 0.66666667,  0.        , -0.66666667,  0.        , -0.33333333,
         0.        ],
       [ 0.        ,  0.66666667,  0.        , -0.66666667,  0.        ,
        -0.33333333],
       [ 0.33333333,  0.        ,  0.66666667,  0.        , -0.66666667,
         0.        ],
       [ 0.        ,  0.33333333,  0.        ,  0.66666667,  0.        ,
        -0.66666667],
       [ 0.66666667,  0.        ,  0.33333333,  0.        ,  0.66666667,
         0.        ]])

In [5]:
L = 10
np.allclose(correlation_mat(Ising_mat(L)), Ising_correlation_mat(L))

True

## Function: ground_state_energy(M)
Input: the matrix $M$

Output: the ground state energy of $H$

In [8]:
ground_state_energy(Ising_mat(L = 3))

-4.0

In [12]:
def Ising_H_def(L, g):
    mat = np.zeros([2**L,2**L])
    # Pauli Matrices
    sigma_x = np.array([[0,1],[1,0]])
    sigma_z = np.array([[1,0],[0,-1]])
    id2 = np.array([[1,0],[0,1]])
    for i in range(L):
        ## X_i X_i+1
        if i == 0 or i == L-1:
            XX = sigma_x
        else:
            XX = id2
        for j in range(1,L):
            if j == i or j == i+1:
                XX = np.kron(XX, sigma_x)
            else:
                XX = np.kron(XX, id2)  
        ## Z_i
        if i == 0:
            Z = sigma_z
        else: 
            Z = id2
        for j in range(1,L):
            if j == i:
                Z = np.kron(Z,sigma_z)
            else:
                Z = np.kron(Z,id2)

        mat -= (XX + g*Z)
    return mat

In [13]:
eigv = LA.eigvalsh(Ising_H_def(L=3, g=1))
print(eigv[0])

-4.0
